In [1]:
import kagglehub

train_path = kagglehub.dataset_download("shreyansjain04/ai-vs-real-image-dataset")

test_path = kagglehub.dataset_download("shreyansjain04/ai-vs-real-image-test-dataset")

/home/andreas/paddy/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import cv2
import numpy as np

def extract_ct(image, kernel_size=3, max_iter=10):
    def em_channel(channel):
        alpha = kernel_size // 2
        padded = cv2.copyMakeBorder(channel, alpha, alpha, alpha, alpha, cv2.BORDER_REFLECT)
        h, w = channel.shape
        N = h * w
        d = kernel_size**2 - 1

        patch_offsets = []
        center = kernel_size // 2
        for i in range(kernel_size):
            for j in range(kernel_size):
                if i == center and j == center:
                    continue
                patch_offsets.append((i - center, j - center))

        A = np.zeros((N, d), dtype=np.float32)
        b = np.zeros(N, dtype=np.float32)

        idx = 0
        for y in range(alpha, h + alpha):
            for x in range(alpha, w + alpha):
                A[idx] = [padded[y + dy, x + dx] for dy, dx in patch_offsets]
                b[idx] = padded[y, x]
                idx += 1

        k = np.zeros(d, dtype=np.float32)
        for _ in range(max_iter):
            pred = A @ k
            residuals = b - pred
            sigma2 = np.mean(residuals**2)
            weights = np.exp(-residuals**2 / (2 * sigma2))

            Aw = A * weights[:, np.newaxis]
            bw = b * weights
            k = np.linalg.pinv(A.T @ Aw) @ (A.T @ bw)

        return k

    if image.shape[2] != 3:
        raise ValueError("Image must be RGB")

    image = image.astype(np.float32) / 255.0
    return np.concatenate([em_channel(image[..., c]) for c in range(3)])


In [ ]:
import cv2
import numpy as np

from fastai.vision.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from pathlib import Path

def get_balanced_subset(path, max_per_class=500):
    files = get_image_files(path)
    grouped = {}
    for f in files:
        lbl = parent_label(f).lower()
        grouped.setdefault(lbl, []).append(f)

    selected = []
    for lbl, f_list in grouped.items():
        selected.extend(f_list[:max_per_class])
    
    return selected

path = Path("sml")

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    get_y=parent_label,
    splitter=RandomSplitter(seed=42)
)

dls = dblock.dataloaders(path, bs=16)

from tqdm import tqdm

i = 0
X, y = [], []
for img, label in tqdm(dls.train_ds):
    try:
        ct_vec = extract_ct(np.array(img), kernel_size=3)
        X.append(ct_vec)
        y.append(int(label))
    except ValueError:
        print(img)
        print("Image not RGB")

    i += 1
    if i % 10000 == 0:
        np.save(f"ct_vectors_{i}.npy", X)

X_valid, y_valid = [], []
for img, label in tqdm(dls.valid_ds):
    try:
        ct_vec = extract_ct(np.array(img), kernel_size=3)
        X_valid.append(ct_vec)
        y_valid.append(int(label))
    except ValueError:
        print("Image not RGB")


rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)

y_pred = rf.predict(X_valid)
print(classification_report(y_valid, y_pred, target_names=["ai", "real"]))

 18%|█▊        | 8837/48096 [2:33:28<12:36:58,  1.16s/it]/tmp/ipykernel_1411/1378728879.py:36: RuntimeWarning: invalid value encountered in divide
  weights = np.exp(-residuals**2 / (2 * sigma2))


PILImage mode=RGB size=128x128
Image not RGB


 12%|█▏        | 1449/12024 [25:02<2:26:42,  1.20it/s]

Image not RGB


 68%|██████▊   | 8150/12024 [2:22:15<1:07:37,  1.05s/it]


KeyboardInterrupt: 